# Lab 06: Credit Card Fraud Detection

---
author: Yiran Hu
date: March 31, 2024
embed-resources: true
---

## Introduction

## Methods

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from joblib import dump


# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

### Data

In [2]:
fraud = pd.read_csv("https://cs307.org/lab-06/data/fraud.csv")
fraud_train, fraud_test = train_test_split(
    fraud,
    test_size=0.20,
    random_state=42,
    stratify=fraud["Fraud"],
)

# create X and y for train
X_train = fraud_train.drop("Fraud", axis=1)
y_train = fraud_train["Fraud"]

# create X and y for test
X_test = fraud_test.drop("Fraud", axis=1)
y_test = fraud_test["Fraud"]

In [4]:
fraud_train

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10,...,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,Amount,Fraud
57638,-0.514509,0.899378,1.627215,-0.142250,0.005250,-0.235422,0.482540,0.247403,-0.562327,-0.166813,...,-0.143290,-0.390205,0.030719,0.184779,-0.348711,0.073253,0.273217,0.107938,3.59,0
27686,-0.813568,-0.373893,1.152977,-0.449774,-3.868866,2.780636,3.654192,-0.672442,0.753230,-0.662803,...,-0.376783,-0.004239,0.074801,0.124238,-0.448493,0.861423,-0.093639,-0.711632,798.01,0
13321,-2.443142,3.258831,-0.791511,0.223548,0.007932,-1.263044,1.220214,-0.418068,1.860453,4.184883,...,-0.348587,0.531679,0.058990,0.371638,-0.207398,-0.505837,0.524542,-0.343895,1.79,0
28308,-0.397300,0.922104,1.224699,-0.334974,0.322603,-0.117372,0.534683,0.175550,-0.486404,-0.120147,...,-0.239303,-0.695001,-0.128231,-0.536463,-0.138971,0.107526,0.255644,0.100814,2.69,0
54252,1.994046,-0.367813,-0.462867,0.338661,-0.485326,-0.241576,-0.590987,0.089319,1.413224,-0.149292,...,-0.196388,-0.484457,0.421867,0.601393,-0.448014,-0.646256,0.027632,-0.027244,4.49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49499,-0.033583,-0.487672,1.435406,-2.759369,-1.785638,0.402364,-2.306038,-2.287179,0.129717,-1.036412,...,-1.286823,0.824100,0.018890,0.000018,0.301618,0.081215,0.163422,0.252456,10.00,0
29250,-0.731245,1.151677,0.912393,-0.653540,0.700577,-0.461372,1.093273,-0.126254,-0.339171,-0.467356,...,-0.368185,-0.991854,-0.376461,-0.661608,0.554237,0.450650,0.076395,0.096252,6.99,0
31586,2.135923,-0.714182,-1.842502,-0.587267,0.090754,-0.465733,-0.024834,-0.328747,-0.659832,0.853241,...,-0.537267,-1.081138,0.196589,0.087797,-0.111583,0.479888,-0.082523,-0.053305,64.90,0
14898,-0.335632,0.939736,-2.961515,-1.224739,4.446891,2.352092,0.789725,0.609460,-0.746845,-1.580751,...,-0.002116,0.009686,-0.428325,0.536043,0.278736,0.680976,-0.069295,0.051891,0.76,0


#### Summary Statistics

In [10]:
# Counts and Proportions
fraud_train_count=fraud_train.groupby('Fraud').agg('count').reset_index()
fraud_train_count['Proportion']=fraud_train_count['Amount']/54276
fraud_train_count=fraud_train_count[['Fraud','Proportion','Amount']]
fraud_train_count

,Fraud,Proportion,Amount
0,0,0.994196,53961
1,1,0.005804,315


In [18]:
# Amount
amount_mean=fraud_train.groupby('Fraud').agg('mean').reset_index()
amount_mean['Mean']=amount_mean['Amount']
print(amount_mean[['Fraud','Mean']], '\n', 'Total Mean: ',fraud_train['Amount'].mean())

   Fraud        Mean
0      0   88.065104
1      1  110.947016 
 Total Mean:  88.19790275628269


In [19]:
amount_std=fraud_train.groupby('Fraud').agg('std').reset_index()
amount_std['Std']=amount_std['Amount']
print(amount_std[['Fraud','Std']], '\n', 'Total Std: ',fraud_train['Amount'].std())

   Fraud         Std
0      0  241.451144
1      1  254.978960 
 Total Std:  241.53561659607715


In [20]:
amount_median=fraud_train.groupby('Fraud').agg('median').reset_index()
amount_median['Median']=amount_median['Amount']
print(amount_median[['Fraud','Median']], '\n', 'Total Median: ',fraud_train['Amount'].median())

   Fraud  Median
0      0   21.80
1      1    6.99 
 Total Median:  21.69


In [21]:
amount_maximum=fraud_train.groupby('Fraud').agg('max').reset_index()
amount_maximum['Maximum']=amount_maximum['Amount']
print(amount_maximum[['Fraud','Maximum']], '\n', 'Total Maximum: ',fraud_train['Amount'].max())

   Fraud   Maximum
0      0  10199.44
1      1   2125.87 
 Total Maximum:  10199.44


In [ ]:
# visualizations


### Models

In [25]:
# split the data into numerical and categorical
numeric_features = X_train.select_dtypes(include=["float64"]).columns
categorical_features = X_train.select_dtypes(include=["object"]).columns

In [26]:
# define how to handle missing data and scale features for numeric and categorical features
numeric_transformer = Pipeline(
    steps=[
        ("Iterative Imputer", IterativeImputer(estimator=LinearRegression())),
        ("Standardization", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("Modal Imputer", SimpleImputer(strategy="most_frequent")),
        ("One-Hot Encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

In [27]:
# create general preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("Numeric Transformer", numeric_transformer, numeric_features),
        ("Categorical Transformer", categorical_transformer, categorical_features),
    ],
    remainder="drop",
)

In [32]:
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', KNeighborsRegressor())])

In [33]:
param_grid = {
    "regressor__n_neighbors": range(1, 30),  # set the range of k
    # define how the neighbors contribute, equally or have different weights based on distance
    "regressor__weights": ["uniform", "distance"],
    # distance metrics to be used for calculating the proximity between data points
    "regressor__metric": ["euclidean", "manhattan", "chebyshev"]
}


In [36]:
# setup grid search with cross-validation
grid_search = GridSearchCV(model_pipeline, param_grid,
                           cv=10, scoring="precision", verbose=1)

In [39]:
# fit models
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 174 candidates, totalling 1740 fits


KeyboardInterrupt: 

## Results

In [ ]:
# report model metrics

## Discussion

### Conclusion